<a href="https://colab.research.google.com/github/lee-jaewook/2021_ai_project_house_price_prediction/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인공신경망을 이용한 주택가격지수 예측

In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import plot_model
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score
import tensorflow as tf
import pydot
import graphviz 
from numpy import unique
from numpy import argmax
import math
from imblearn.over_sampling import SMOTE

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AI 2021/Dataset/house_dataset(micro_macro).csv")

In [5]:
for i in range(df.shape[0]):
    df["Date"][i]=df["Date"][i][0:4]+df["Date"][i][6:-1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [115]:
X = np.array(df.iloc[:,1:-1])
y = np.array(df.iloc[:,-1])

In [116]:
X.shape

(168, 12)

In [117]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X=scaler.transform(X)

### Train : 0~143( ~ 2018)
### test : 144~ (2019 ~)

In [135]:
X_macro_train=X[:144,:6]
X_macro_test=X[144:,:6]
X_micro_train=X[:144,6:]
X_micro_test=X[144:,6:]

y_train = y[:144]
y_test = y[144:]

# Macro data (CASE 1)
---

In [ ]:
look_back = 6
trainX = np.reshape(X_macro_train, (X_macro_train.shape[0], 1, X_macro_train.shape[1]))
testX = np.reshape(X_macro_test, (X_macro_test.shape[0], 1, X_macro_test.shape[1]))

model = Sequential()
model.add(LSTM(64, input_shape=(1, look_back)))
# model.add(LSTM(4, input_shape=(1, look_back), return_sequences = True))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, y_train, epochs=1000, batch_size=10, verbose=2)

# evaluate on test set
preds = model.predict(testX)

preds1=np.reshape(preds,(preds.shape[0],))

case1 = np.concatenate((y_train,preds1),axis=0)

tf.keras.backend.clear_session()

Epoch 1/1000
15/15 - 1s - loss: 7110.5894
Epoch 2/1000
15/15 - 0s - loss: 7095.9849
Epoch 3/1000
15/15 - 0s - loss: 7078.9297
Epoch 4/1000
15/15 - 0s - loss: 7058.6230
Epoch 5/1000
15/15 - 0s - loss: 7032.1543
Epoch 6/1000
15/15 - 0s - loss: 7000.5913
Epoch 7/1000
15/15 - 0s - loss: 6961.2979
Epoch 8/1000
15/15 - 0s - loss: 6913.7319
Epoch 9/1000
15/15 - 0s - loss: 6858.7842
Epoch 10/1000
15/15 - 0s - loss: 6796.8062
Epoch 11/1000
15/15 - 0s - loss: 6727.2681
Epoch 12/1000
15/15 - 0s - loss: 6652.4883
Epoch 13/1000
15/15 - 0s - loss: 6572.3345
Epoch 14/1000
15/15 - 0s - loss: 6487.8301
Epoch 15/1000
15/15 - 0s - loss: 6399.7192
Epoch 16/1000
15/15 - 0s - loss: 6310.6475
Epoch 17/1000
15/15 - 0s - loss: 6217.1011
Epoch 18/1000
15/15 - 0s - loss: 6123.4102
Epoch 19/1000
15/15 - 0s - loss: 6029.2446
Epoch 20/1000
15/15 - 0s - loss: 5933.5498
Epoch 21/1000
15/15 - 0s - loss: 5837.6016
Epoch 22/1000
15/15 - 0s - loss: 5741.4609
Epoch 23/1000
15/15 - 0s - loss: 5643.9985
Epoch 24/1000
15/15 

# Micro data (CASE 2)
---

In [ ]:
look_back = 6
trainX = np.reshape(X_micro_train, (X_micro_train.shape[0], 1, X_micro_train.shape[1]))
testX = np.reshape(X_micro_test, (X_micro_test.shape[0], 1, X_micro_test.shape[1]))

model = Sequential()
model.add(LSTM(64, input_shape=(1, look_back)))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, y_train, epochs=1000, batch_size=10, verbose=2)

# evaluate on test set
preds = model.predict(testX)

preds2=np.reshape(preds,(preds.shape[0],))

case2 = np.concatenate((y_train,preds2),axis=0)

tf.keras.backend.clear_session()

# All data (CASE 3)
---

In [ ]:
look_back = 12
X_train=X[:144,]
X_test=X[144:,]
trainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
testX = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

model = Sequential()
model.add(LSTM(64, input_shape=(1, look_back)))
model.add(Dense(1, activation='linear'))
model.compile(loss='mse', optimizer='adam')
model.fit(trainX, y_train, epochs=1000, batch_size=10, verbose=2)

# evaluate on test set
preds = model.predict(testX)

preds3=np.reshape(preds,(preds.shape[0],))

case3 = np.concatenate((y_train,preds3),axis=0)

tf.keras.backend.clear_session()

# matplot graph

In [ ]:
plt.figure(figsize=(12, 6))
x_date = df.iloc[:,0]
plt.plot(x_date,case1, color="salmon")
plt.plot(x_date,case2, color="seagreen")
plt.plot(x_date,case3, color="deeppink")
plt.plot(x_date,y, color="black")
plt.legend(['case 1','case 2','case 3', 'real'])
plt.xticks(ticks=x_date, rotation=45)
plt.locator_params(axis='x', nbins=x_date.shape[0]/12)
plt.show()

In [ ]:
arr=[mean_squared_error(y_test,preds1), mean_squared_error(y_test,preds2), mean_squared_error(y_test,preds3)]
result=pd.DataFrame(arr).transpose()

In [ ]:
result

In [ ]:
import os
if not os.path.exists('/content/drive/MyDrive/Colab Notebooks/AI 2021/result_lstm.csv'):
    result.to_csv('/content/drive/MyDrive/Colab Notebooks/AI 2021/result_lstm.csv', index=False, mode='w', encoding='utf-8-sig')
else:
    result.to_csv('/content/drive/MyDrive/Colab Notebooks/AI 2021/result_lstm.csv', index=False, mode='a', encoding='utf-8-sig', header=False)